Google Vertex

In this quickstart you will learn how to run evaluation functions using models from google Vertex like PaLM-2.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/google_vertex_quickstart.ipynb)

In [1]:
#! pip install google-cloud-aiplatform==1.36.3 litellm==0.14.1 trulens_eval==0.17.0 langchain==0.0.323

## Setup
### Add API keys
For this quickstart you will need an OpenAI key.

In [2]:
from google.cloud import aiplatform

In [3]:
aiplatform.init(
    project = "...",
    location="us-central1"
)

### Import from LangChain and TruLens

In [4]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru, LiteLLM
tru = Tru()
tru.reset_database()


# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import VertexAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Deleted 4 rows.


### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [5]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = VertexAI()

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

### Send your first request

In [6]:
prompt_input = 'What is a good name for a store that sells colorful socks?'

In [7]:
llm_response = chain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: What is a good name for a store that sells colorful socks?

> Finished chain.


{'prompt': 'What is a good name for a store that sells colorful socks?',
 'text': ' The sock drawer is a great name for a store that sells colorful socks because it is a place where people can find a variety of socks to match their personality and style. The name is also memorable and easy to spell, which will help customers remember the store when they are looking for a place to buy socks.\n\nHere are some other ideas for names for a store that sells colorful socks:\n\n* The sock exchange\n* The sock market\n* The sock shop\n* The sock emporium\n* The sock palace\n* The sock world\n* The sock universe\n* The sock galaxy\n* The sock multiverse\n* The sock'}

## Initialize Feedback Function(s)

In [8]:
# Initialize LiteLLM-based feedback function collection class:
litellm = LiteLLM(model_engine="chat-bison")

# Define a relevance function using LiteLLM
relevance = Feedback(litellm.relevance_with_cot_reasons).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

✅ In relevance_with_cot_reasons, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance_with_cot_reasons, input response will be set to __record__.main_output or `Select.RecordOutput` .


## Instrument chain for logging with TruLens

In [9]:
tru_recorder = TruChain(chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[relevance])

✅ added app Chain1_ChatApplication
✅ added feedback definition feedback_definition_hash_25d06d2c03927fdd8415a39bab8485d8


In [10]:
with tru_recorder as recording:
    llm_response = chain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: What is a good name for a store that sells colorful socks?

> Finished chain.
✅ added record record_hash_4520ac2beab2879528b2ab24fd23a37d


{'prompt': 'What is a good name for a store that sells colorful socks?',
 'text': ' The sock drawer is a great name for a store that sells colorful socks because it is a place where people can find a variety of socks to match their personality and style. The name is also memorable and easy to spell, which will help customers remember the store when they are looking for a place to buy socks.\n\nHere are some other ideas for names for a store that sells colorful socks:\n\n* The sock exchange\n* The sock market\n* The sock shop\n* The sock emporium\n* The sock palace\n* The sock world\n* The sock universe\n* The sock galaxy\n* The sock multiverse\n* The sock'}

In [11]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""app_id"": ""Chain1_ChatApplication"", ""tags"": ""...",LLMChain(langchain.chains.llm),record_hash_4520ac2beab2879528b2ab24fd23a37d,"""What is a good name for a store that sells co...",""" The sock drawer is a great name for a store ...",-,"{""record_id"": ""record_hash_4520ac2beab2879528b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-11-15T21:52:06.287839"", ""...",2023-11-15T21:52:08.399417,2,0,0.0


## Explore in a Dashboard

In [12]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.16.0.36:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [13]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""app_id"": ""Chain1_ChatApplication"", ""tags"": ""...",LLMChain(langchain.chains.llm),record_hash_4520ac2beab2879528b2ab24fd23a37d,"""What is a good name for a store that sells co...",""" The sock drawer is a great name for a store ...",-,"{""record_id"": ""record_hash_4520ac2beab2879528b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-11-15T21:52:06.287839"", ""...",2023-11-15T21:52:08.399417,2,0,0.0


✅ feedback result relevance_with_cot_reasons DONE feedback_result_hash_05c9037be111ac62af31f9325c851346
